In [3]:
import yfinance as yf
from langchain import PromptTemplate, LLMChain
from langchain.chat_models import ChatOpenAI
# from langchain.tools import DuckDuckGoSearchAPIWrapper
import requests
from bs4 import BeautifulSoup
import csv
from io import StringIO

# Initialize ChatOpenAI
llm = ChatOpenAI(
    temperature=0.1,
    model="gpt-4o-mini",
)

# Initialize DuckDuckGo search
# search = DuckDuckGoSearchAPIWrapper()

class Tools:
    @staticmethod
    def stock_price(ticker):
        stock = yf.Ticker(ticker)
        return stock.history(period="1mo").to_csv()

    @staticmethod
    def stock_news(ticker):
        stock = yf.Ticker(ticker)
        return list(map(lambda x: x["link"], stock.news))

    @staticmethod
    def income_stmt(ticker):
        stock = yf.Ticker(ticker)
        return stock.income_stmt.to_csv()

    @staticmethod
    def balance_sheet(ticker):
        stock = yf.Ticker(ticker)
        return stock.balance_sheet.to_csv()

    @staticmethod
    def insider_transactions(ticker):
        stock = yf.Ticker(ticker)
        return stock.insider_transactions.to_csv()

    @staticmethod
    def scrape_website(url):
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        return soup.get_text()

def technical_analysis(ticker):
    stock_data = Tools.stock_price(ticker)
    template = """
    You are a Technical Analyst. Your goal is to analyze the movements of a stock and provide insights on trends, entry points, resistance and support levels.
    Given the following stock price data for the past month, provide your analysis:

    {stock_data}

    Your analysis should include:
    1. Identification of key trends (bullish, bearish, or sideways)
    2. Support and resistance levels
    3. Any notable chart patterns
    4. Potential entry and exit points
    5. Technical indicators (e.g., moving averages, RSI, MACD) and their implications

    Analysis:
    """
    prompt = PromptTemplate(template=template, input_variables=["stock_data"])
    llm_chain = LLMChain(prompt=prompt, llm=llm)
    return llm_chain.run(stock_data=stock_data)

def research_analysis(ticker):
    news_urls = Tools.stock_news(ticker)
    news_contents = [Tools.scrape_website(url) for url in news_urls[:3]]  # Limit to first 3 for brevity
    
    template = """
    You are a Researcher. Your goal is to gather, interpret and summarize vast amounts of data to provide a comprehensive overview of the sentiment and news surrounding a stock.
    Given the following news articles, provide your analysis:

    {news_contents}

    Your analysis should include:
    1. Summary of key news events
    2. Overall market sentiment (positive, negative, or neutral)
    3. Potential impact on stock price
    4. Any notable shifts in market perception
    5. Identification of potential risks or opportunities based on the news

    Analysis:
    """
    prompt = PromptTemplate(template=template, input_variables=["news_contents"])
    llm_chain = LLMChain(prompt=prompt, llm=llm)
    return llm_chain.run(news_contents="\n\n".join(news_contents))

def financial_analysis(ticker):
    income_stmt = Tools.income_stmt(ticker)
    balance_sheet = Tools.balance_sheet(ticker)
    insider_transactions = Tools.insider_transactions(ticker)
    
    template = """
    You are a Financial Analyst. Your goal is to use financial statements, insider trading data, and other financial metrics to evaluate a stock's financial health and performance.
    Given the following financial data, provide your analysis:

    Income Statement:
    {income_stmt}

    Balance Sheet:
    {balance_sheet}

    Insider Transactions:
    {insider_transactions}

    Your analysis should include:
    1. Overview of the company's financial health
    2. Key financial metrics and their trends (e.g., revenue growth, profit margins, debt levels)
    3. Analysis of insider transactions and their potential implications
    4. Comparison to industry averages (if possible)
    5. Identification of financial strengths and weaknesses

    Analysis:
    """
    prompt = PromptTemplate(template=template, input_variables=["income_stmt", "balance_sheet", "insider_transactions"])
    llm_chain = LLMChain(prompt=prompt, llm=llm)
    return llm_chain.run(income_stmt=income_stmt, balance_sheet=balance_sheet, insider_transactions=insider_transactions)

def hedge_fund_manager_decision(ticker, technical_analysis, research_analysis, financial_analysis):
    template = """
    You are a Hedge Fund Manager. Your goal is to manage a portfolio of stocks and make strategic investment decisions to maximize returns using insights from financial analysts, technical analysts, and researchers.
    Given the following analyses from your team, provide your final decision on whether to buy, sell, or hold the stock {ticker}:

    Technical Analysis:
    {technical_analysis}

    Research Analysis:
    {research_analysis}

    Financial Analysis:
    {financial_analysis}

    Your decision should include:
    1. Clear buy, sell, or hold recommendation
    2. Rationale for your decision, synthesizing insights from all analyses
    3. Potential risks and how they might be mitigated
    4. Expected return and time horizon for the investment
    5. Any conditions or events that might change your recommendation

    Final Decision:
    """
    prompt = PromptTemplate(template=template, input_variables=["ticker", "technical_analysis", "research_analysis", "financial_analysis"])
    llm_chain = LLMChain(prompt=prompt, llm=llm)
    return llm_chain.run(ticker=ticker, technical_analysis=technical_analysis, research_analysis=research_analysis, financial_analysis=financial_analysis)

def analyze_stock(ticker):
    tech_analysis = technical_analysis(ticker)
    research_analysis_result = research_analysis(ticker)
    fin_analysis = financial_analysis(ticker)

    final_decision = hedge_fund_manager_decision(ticker, tech_analysis, research_analysis_result, fin_analysis)

    return final_decision

# Example usage
ticker = "AAPL"  # Replace with the desired stock ticker
result = analyze_stock(ticker)
print(result)

/var/folders/gh/6gsh3jws6nx9x0d7wq7516p40000gn/T/ipykernel_13295/808759675.py:11: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm = ChatOpenAI(
/var/folders/gh/6gsh3jws6nx9x0d7wq7516p40000gn/T/ipykernel_13295/808759675.py:69: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  llm_chain = LLMChain(prompt=prompt, llm=llm)
/var/folders/gh/6gsh3jws6nx9x0d7wq7516p40000gn/T/ipykernel_13295/808759675.py:70: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  return llm_chain.run(stock_data=stock_data)


### Final Decision: Hold

#### Rationale for Decision:
1. **Technical Analysis**: The stock has shown a bullish trend followed by a bearish correction and consolidation. Key support levels at $220 and $222 indicate potential areas for a bounce back, while resistance at $229 suggests a ceiling that has not been broken recently. The presence of a double top pattern raises concerns about a potential reversal, indicating that caution is warranted. The technical indicators, such as moving averages and RSI, should be closely monitored for further signals.

2. **Research Analysis**: The positive sentiment surrounding Affirm Holdings (AFRM) and its strategic initiatives may indirectly impact AAPL, especially if AAPL continues to integrate with fintech solutions. However, the overall market sentiment for AAPL is mixed, with potential growth opportunities tempered by competitive pressures and economic uncertainties.

3. **Financial Analysis**: AAPL's recent decline in revenue and net income, cou